## Homework3: Vector Search

### Q1. Getting the embeddings model

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
embedding_model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(embedding_model_name)

In [ ]:
user_question = "I just discovered the course. Can I still join it?"
query_vector = embedding_model.encode(user_question)

In [ ]:
query_vector[0]

### Q2. Creating the embeddings

In [ ]:
import requests 

In [ ]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [ ]:
filtered_documents = [item for item in documents if item.get('course') == 'machine-learning-zoomcamp']

In [ ]:
from tqdm.auto import tqdm

In [ ]:
embeddings = []
for doc in tqdm(filtered_documents):
    qa_text = f'{doc["question"]} {doc["text"]}'
    
    # Transforming the qa_text into an embedding using the embedding_model
    # doc["qa_text_vector"] = embedding_model.encode(qa_text).tolist()
    # embeddings.append(doc)
    embeddings.append(embedding_model.encode(qa_text))

In [ ]:
import numpy as np
X = np.array(embeddings)
X.shape

### Q3. Search

In [ ]:
v = query_vector
v.shape

In [ ]:
v.dot(v)

In [ ]:
X[0].dot(X[0])

In [ ]:
scores = X.dot(query_vector)
scores.max()

### Q4. Hit-rate for our search engine

In [ ]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [ ]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)
results = search_engine.search(query_vector, num_results=5)

In [ ]:
import pandas as pd

In [ ]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [ ]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
def evaluate(ground_truth, search_function, embedding_model):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = embedding_model.encode(q['question'])
        results = search_function(v)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
evaluate(ground_truth, search_engine.search, embedding_model)

### Q5. Indexing with Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for i,doc in tqdm(enumerate(filtered_documents)):
    doc['question_text_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)

In [ ]:
es_client.count(index=index_name)

In [ ]:
def elastic_search_knn_small(query_vector):
    search_query = {
        "knn": {
            "field": 'question_text_vector',
            "query_vector": query_vector,
            "k": 5,
            "num_candidates": 10000,
        },
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
search_query = {
    "knn": {
        "field": 'question_text_vector',
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 10000,
    },
    "_source": ["text", "section", "question", "course", "id"]
}

In [ ]:
es_results = es_client.search(
        index=index_name,
        body=search_query
    )

In [ ]:
es_results['hits']['hits'][0]['_score']